# Clustering Neighborhoods in Toronto

## Please Note: This notebook contains maps that are not visible on github since you must 'Trust the Notebook'.

Follow this link to view the maps as well as the rest of the notebook:  
https://nbviewer.jupyter.org/github/mingusknoxwhite/Coursera_Capstone/blob/main/scripts/TorontoNeighborhoods.ipynb

Download and import any libraries we'll need.

In [1]:
#import sys
#!{sys.executable} -m pip install matplotlib
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import geocoder
import folium

import requests
import json

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

from IPython.display import display 

### Part 1: Create DataFrame of Toronto, Ontario Postal Codes

Save the url we will use to get our Toronto neighborhood data.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

Read the url and isolate the data we're interested in.

In [3]:
data = pd.read_html(url)
df = data[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We are not intereseted in 'Not assigned' boroughs, so we can remove the rows that include them.
Reset the index since we removed rows.

In [4]:
df['Borough'].replace('Not assigned', np.nan, inplace = True)
df.dropna(subset=['Borough'], inplace = True)
df.reset_index(inplace=True, drop = True)
df.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Check the shape of the dataframe.

In [5]:
df.shape

(103, 3)

So there are 103 Postal Codes in our dataframe.

### Part 2: Add Latitude and Longitude Coordinates of Postal Codes to DataFrame

Use the geocoder package to get the Latitude and Longitude coordinates for each post code

In [6]:
latitude = []
longitude = []
for i in range(df.shape[0]):
    # package can sometimes give a None result, but will eventually give coords when rerun
    lat_lng_coords = None
    while (lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(df['Postal Code'][i]))
        lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

Add the Latitude and Longitude coords to the dataframe.

In [7]:
df['Latitude'] = latitude
df['Longitude'] = longitude
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


### Part 3: Explore and Cluster Neighborhoods in Toronto

*Note: We want to use Postal Codes to label and group our data because some include multiple neighborhoods, which would cause labels to be more confusing.*

Plot the postal codes before performing any analysis.

In [8]:
# Initialize map centered at average coordinates of neighborhoods
avg_lat = df['Latitude'].mean()
avg_lng = df['Longitude'].mean()
map_toronto = folium.Map(location = [avg_lat, avg_lng], zoom_start = 11)

# Add the postal codes to the map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Postal Code']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat, lng],
                        radius = 3, 
                        popup = label, 
                        color = 'blue', 
                        fill = True,
                        fill_color = 'blue', 
                        fill_opacity = 0.7, 
                        parse_html = False).add_to(map_toronto)

map_toronto

#### *To explore the neighborhoods, we will use the Foursquare API.*

Import Foursquare credentials from another file for confidentiality. We will use the credentials file in the same directory.

In [9]:
from credentials import foursquare

Define Credentials and version to use the API

In [10]:
CLIENT_ID = foursquare['id']
CLIENT_SECRET = foursquare['secret']
VERSION = '20201201'
LIMIT = 100

#### Goal: Explore venues around each neighborhood and onehot encode them for clustering

Define a function to return the category of a venue

In [11]:
def get_venues(postals, lats, lngs, radius = 500):
    
    # Create list for all venues from all neighborhoods
    venues_list = []
    for postal, lat, lng in zip(postals, lats, lngs):
        # Create the API request URL that will be used to explore each neighborhood
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        # GET request to get json data
        try:
            results = requests.get(url).json()['response']['groups'][0]['items']
        
            # Add relevant information about venues to venues_list
            for venue in results:
                venues_list.append([
                    postal,
                    lat,
                    lng,
                    venue['venue']['name'],
                    venue['venue']['categories'][0]['name']
                ])
        except:
            pass
        
    venues_df = pd.DataFrame(venues_list)
    venues_df.columns = ['Postal Code', 'Latitude', 'Longitude', 'Venue', 'Venue Category']
    
    return(venues_df)

Apply get_venues function to our Toronto neighborhood data

In [12]:
toronto_venues = get_venues(postals = df['Postal Code'], lats = df['Latitude'], lngs = df['Longitude'])

In [13]:
print(toronto_venues.shape)
toronto_venues.head()

(2375, 5)


,Postal Code,Latitude,Longitude,Venue,Venue Category
0,M3A,43.75245,-79.32991,Brookbanks Park,Park
1,M3A,43.75245,-79.32991,Variety Store,Food & Drink Shop
2,M4A,43.73057,-79.31306,Wigmore Park,Park
3,M4A,43.73057,-79.31306,Memories of Africa,Grocery Store
4,M4A,43.73057,-79.31306,Vinnia Meats,German Restaurant


One-hot encode the venue categories into a new DataFrame.

In [14]:
onehot_toronto = pd.get_dummies(toronto_venues['Venue Category'])
onehot_toronto['Postal Code'] = toronto_venues['Postal Code']
col_order = [onehot_toronto.columns[-1]] + list(onehot_toronto.columns[:-1])
onehot_toronto = onehot_toronto[col_order]
onehot_toronto.head()

,Postal Code,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Check the shape of our new df. There should be 1 more column than there are unique categories since we added 'Postal Code' to our df.

In [15]:
onehot_toronto.shape

(2375, 264)

In [16]:
len(toronto_venues['Venue Category'].unique())

263

Group the encoded data by Postal Code, taking the mean of each encoded column. This will allow us to find the most common venue categories for each Postal Code.

In [17]:
toronto_grouped = onehot_toronto.groupby('Postal Code').mean().reset_index()
toronto_grouped.head()

,Postal Code,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Peking Duck Restaurant,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.

Check the shape of the grouped dataframe to make sure data was properly grouped.

In [18]:
toronto_grouped.shape

(101, 264)

In [19]:
len(onehot_toronto['Postal Code'].unique())

101

*Note: There may be Postal Codes absent from the encoded dataframes that were present in the original dataframe of all Postal Codes.*

In [20]:
if len(df[~df['Postal Code'].isin(toronto_grouped['Postal Code'])]) == 0:
    print("All Postal Codes are accounted for.")
else:
    print("These are the missing Postal Codes.\n")
    print(df[~df['Postal Code'].isin(toronto_grouped['Postal Code'])])

These are the missing Postal Codes.

   Postal Code      Borough Neighbourhood  Latitude  Longitude
53         M3M   North York     Downsview  43.73224  -79.50178
95         M1X  Scarborough   Upper Rouge  43.83412  -79.21668


These Postal Codes are missing because they had no venues to return from our Foursquare GET request, so we can ignore them.

### Now we'll create a DataFrame with each Postal Code, with latitude, longitude, and top 10 venue categories for each.

In [21]:
num_top = 10
suffixes = ['st', 'nd', 'rd']

# Set column names
cols = ['Postal Code']
for i in range(num_top):
    try:
        cols.append('{}{} Most Common Venue'.format(i+1, suffixes[i]))
    except:
        cols.append('{}th Most Common Venue'.format(i+1))

# Create the df        
top_venues = pd.DataFrame(columns = cols)
top_venues['Postal Code'] = toronto_grouped['Postal Code']

# Get top categories for each Postal Code, setting None if there are less than num_top
for i in range(toronto_grouped.shape[0]):
    category_list = toronto_grouped.iloc[i, 1:]
    sorted_cats = category_list.sort_values(ascending = False)
    
    check_tops = 0
    while (sorted_cats[check_tops] != 0) and (check_tops < num_top):
        check_tops += 1
    
    top_cats = sorted_cats.index.values[:check_tops]
    top_cats = np.append(top_cats, [None for i in range(num_top - check_tops)])
    
    top_venues.iloc[i, 1:] = top_cats

top_venues.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,None,None,None,None,None,None,None,None,None
1,M1C,Construction & Landscaping,Bar,None,None,None,None,None,None,None,None
2,M1E,Construction & Landscaping,Park,Gym / Fitness Center,None,None,None,None,None,None,None
3,M1G,Coffee Shop,Park,Business Service,Korean BBQ Restaurant,None,None,None,None,None,None
4,M1H,Gaming Cafe,Lounge,None,None,None,None,None,None,None,None


Run k-means to cluster the Postal Codes.

In [29]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

kmeans = KMeans(n_clusters = kclusters).fit(toronto_grouped_clustering)
kmeans.labels_

array([1, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 0, 3, 3,
       4, 4, 0, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 0, 4, 3, 3, 3, 4,
       4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3,
       3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 4, 2, 3, 4, 3, 3, 3, 3, 3, 4, 3,
       0, 3, 3, 3, 3, 4, 2, 3, 4, 3, 3, 3, 3], dtype=int32)

Create a DataFrame that contains each Postal Code and its details, cluster number, and top venues.

In [30]:
# add cluster labels to top_venues df, using try/except to allow reruns of cell
try:
    top_venues.insert(0, 'Cluster Label', kmeans.labels_ + 1)
except:
    top_venues['Cluster Label'] = kmeans.labels_ + 1
full_toronto = df.join(top_venues.set_index('Postal Code'), on='Postal Code', how='inner').reset_index().drop('index', 1)
full_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,1,Park,Food & Drink Shop,None,None,None,None,None,None,None,None
1,M4A,North York,Victoria Village,43.73057,-79.31306,5,Park,Pharmacy,German Restaurant,Grocery Store,None,None,None,None,None,None
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,4,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Event Space,Spa,Electronics Store,Food Truck,Restaurant,Distribution Center
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,4,Clothing Store,Men's Store,Toy / Game Store,Cosmetics Shop,American Restaurant,Women's Store,Furniture / Home Store,Bookstore,Restaurant,Food Court
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,4,Coffee Shop,Sandwich Place,Café,Gastropub,Mediterranean Restaurant,Fried Chicken Joint,Italian Restaurant,Theater,Intersection,Bank


Plot the Postal Codes again, this time they will be colored and labeled depending on their cluster.

In [31]:
# Initialize the map using the avg_lat, avg_lng variables from the last map
map_clusters = folium.Map(location=[avg_lat, avg_lng], zoom_start = 11)

# set color scheme for clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lng, post, clust in zip(full_toronto['Latitude'], full_toronto['Longitude'], full_toronto['Postal Code'], full_toronto['Cluster Label']):
    label = folium.Popup(str(post) + ' Cluster ' + str(clust), parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius = 5,
                        popup = label,
                        color = rainbow[clust - 1],
                        fill = True,
                        fill_color = rainbow[clust - 1],
                        fill_opacity = 0.7).add_to(map_clusters)

map_clusters

### The Neighbourhood column often includes multiple neighbourhoods per Postal Code, so we have used the Postal Codes for labels and grouping to improve the readability. 

#### The next chunks of code will allow you to search for rows based on an user input of Postal Code or Neighborhood.

#### Search by Postal Code

In [32]:
input_post = input("Which Postal Code(s) are you interested in? (Please separate with commas.) ")
postals = pd.Series([s.strip() for s in input_post.split(',')])
full_toronto[full_toronto['Postal Code'].isin(postals)]

Which Postal Code(s) are you interested in? (Please separate with commas.)  M3A, M4A


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,1,Park,Food & Drink Shop,None,None,None,None,None,None,None,None
1,M4A,North York,Victoria Village,43.73057,-79.31306,5,Park,Pharmacy,German Restaurant,Grocery Store,None,None,None,None,None,None


#### Search by Neighborhood

In [33]:
input_neigh = input("Which Neighbourhood(s) are you interested in? (Please separate with commas.) ")
neighs = [s.strip() for s in input_neigh.split(',')]

### run these lines if you want to search for an exact match to your input
## Next line splits Neighbourhood column on commas to separate individual neighborhoods, checks if any neighborhoods in neighs are in that Neighbourhood column (boolean)
neigh_match = [full_toronto['Neighbourhood'].str.split(',').apply(lambda neighborhoods: neigh in [neighborhood.strip() for neighborhood in neighborhoods]) for neigh in neighs]
# Prints row if any column has a True (neighborhood matches)
full_toronto[np.logical_or.reduce(neigh_match)]

### run this line if you want to search Neighbourhoods for matches beginning with your input
#full_toronto[full_toronto.Neighbourhood.str.contains('|'.join(neighs), na=False)]

Which Neighbourhood(s) are you interested in? (Please separate with commas.)  Rouge, Parkwoods


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,1,Park,Food & Drink Shop,None,None,None,None,None,None,None,None
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662,2,Fast Food Restaurant,None,None,None,None,None,None,None,None,None


Examine the clusters.

In [34]:
# Helpful columns for comparing cluster
important_cols = full_toronto.columns[([0] + list(range(-num_top, 0)))]
list(important_cols)

['Postal Code',
 '1st Most Common Venue',
 '2nd Most Common Venue',
 '3rd Most Common Venue',
 '4th Most Common Venue',
 '5th Most Common Venue',
 '6th Most Common Venue',
 '7th Most Common Venue',
 '8th Most Common Venue',
 '9th Most Common Venue',
 '10th Most Common Venue']

### Print each cluster to distinguish defining factors of each.

It is likely that clusters will change if rerun, but we can examine the differences between the clusters that are defined and printed below.

Cluster 1: Areas with Parks, but not many more venues  
Cluster 2: Areas with only Fast Food Restaurants  
Cluster 3: Areas with Construction and Landscaping venues  
Cluster 4: Areas with Cafes, likely more commercial with Restaurants and Shops  
Cluster 5: Areas with Parks, as well as some Restaurants or Shops

In [35]:
for i in range(1, kclusters + 1):
    print('Cluster ' + str(i))
    display(full_toronto.loc[full_toronto['Cluster Label'] == i, important_cols])

Cluster 1


,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,Park,Food & Drink Shop,None,None,None,None,None,None,None,None
35,M4J,Park,Intersection,None,None,None,None,None,None,None,None
45,M2L,Park,None,None,None,None,None,None,None,None,None
67,M5P,Basketball Court,Park,None,None,None,None,None,None,None,None
96,M8X,Park,Lounge,None,None,None,None,None,None,None,None


Cluster 2


,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1B,Fast Food Restaurant,None,None,None,None,None,None,None,None,None


Cluster 3


,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,M1C,Construction & Landscaping,Bar,None,None,None,None,None,None,None,None
50,M9L,Construction & Landscaping,Rental Car Location,None,None,None,None,None,None,None,None
55,M6M,Construction & Landscaping,Playground,Fast Food Restaurant,Coffee Shop,None,None,None,None,None,None


Cluster 4


,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Coffee Shop,Breakfast Spot,Yoga Studio,Theater,Event Space,Spa,Electronics Store,Food Truck,Restaurant,Distribution Center
3,M6A,Clothing Store,Men's Store,Toy / Game Store,Cosmetics Shop,American Restaurant,Women's Store,Furniture / Home Store,Bookstore,Restaurant,Food Court
4,M7A,Coffee Shop,Sandwich Place,Café,Gastropub,Mediterranean Restaurant,Fried Chicken Joint,Italian Restaurant,Theater,Intersection,Bank
5,M9A,Pharmacy,Park,Café,Shopping Mall,Grocery Store,Skating Rink,Bank,None,None,None
7,M3B,Gas Station,Soccer Field,Burger Joint,Park,Coffee Shop,None,None,None,None,None
8,M4B,Pizza Place,Flea Market,Intersection,Pharmacy,Rock Climbing Spot,Café,Fast Food Restaurant,Breakfast Spot,Gastropub,Bank
9,M5B,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Cosmetics Shop,Bubble Tea Shop,Italian Restaurant,Chinese Restaurant
10,M6B,Grocery Store,Bank,Mediterranean Restaurant,Bakery,Japanese Restaurant,Latin American Restaurant,Pub,Pizza Place,Fast Food Restaurant,Gas Station
11,M9B,Pizza Place,Chinese Restaurant,Sandwich Place,Tea Room,None,None,None,None,None,None
13,M3C,Bubble Tea Shop,Intersection,Beer Store,Gym,Grocery Store,Supermarket,Coffee Shop,None,None,None


Cluster 5


,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,Park,Pharmacy,German Restaurant,Grocery Store,None,None,None,None,None,None
16,M6C,Park,Trail,Field,Hockey Arena,Grocery Store,None,None,None,None,None
17,M9C,Carpet Store,Fish & Chips Shop,College Rec Center,Electronics Store,Grocery Store,Park,Shopping Mall,None,None,None
18,M1E,Construction & Landscaping,Park,Gym / Fitness Center,None,None,None,None,None,None,None
22,M1G,Coffee Shop,Park,Business Service,Korean BBQ Restaurant,None,None,None,None,None,None
32,M1J,Indian Restaurant,Spa,Restaurant,Grocery Store,Park,None,None,None,None,None
39,M2K,Trail,Construction & Landscaping,Golf Driving Range,Park,None,None,None,None,None,None
41,M4K,Ice Cream Shop,Intersection,Bus Line,Park,Grocery Store,Discount Store,Business Service,None,None,None
49,M6L,Bakery,Basketball Court,Park,Trail,None,None,None,None,None,None
63,M9N,Pizza Place,Pharmacy,Diner,Grocery Store,Park,Fried Chicken Joint,None,None,None,None
